In [14]:
import numpy as np
import pandas as pd
from typing import Union
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from module.pricing import DualDigital

# -----
# SET DISPLAY SETTINGS
# -----
pd.set_option('display.width', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

In [2]:
class Backtest:

    def __init__(self, model: Union[None, str], periods_per_year: int = 252):
        self.model = model
        self.periods_per_year = periods_per_year

    def dual_digital(self,
                     data: dict,
                     k1: float, k2: float,
                     iv1: float, iv2: float,
                     q1: float, q2: float,
                     b1: float, b2: float,
                     rho: float, r: float, unit: int):

        # -------
        # COMPUTE PRODUCT PV AND GREEKS TIMESERIES
        # -------

        bt = dict()

        dates = list(data.keys())
        date_t0 = dates[0]

        st1_t0 = data.get(date_t0).get('st1')
        st2_t0 = data.get(date_t0).get('st2')

        business_day_to_expiry = len(dates) - 1
        t = business_day_to_expiry / self.periods_per_year

        for date in tqdm(dates):

            st1 = data.get(date).get('st1')
            st2 = data.get(date).get('st2')

            st1_pct = st1 / st1_t0
            st2_pct = st2 / st2_t0

            priceable_up = DualDigital(
                st1=st1_pct, k1=k1, iv1=iv1, q1=q1, b1=b1, direction1='up',
                st2=st2_pct, k2=k2, iv2=iv2, q2=q2, b2=b2, direction2='up',
                rho=rho, r=r, t=t, unit=unit, model=self.model
            )

            priceable_up.calculate_present_value()
            priceable_up.calculate_greeks()
            pv_up = priceable_up.get_present_value()
            greeks_up = priceable_up.get_greeks()

            priceable_down = DualDigital(
                st1=st1_pct, k1=k1, iv1=iv1, q1=q1, b1=b1, direction1='down',
                st2=st2_pct, k2=k2, iv2=iv2, q2=q2, b2=b2, direction2='down',
                rho=rho, r=r, t=t, unit=unit, model=self.model
            )

            priceable_down.calculate_present_value()
            priceable_down.calculate_greeks()
            pv_down = priceable_down.get_present_value()
            greeks_down = priceable_down.get_greeks()

            pv = pv_up + pv_down
            delta_st1 = (greeks_up.get('dst1') + greeks_down.get('dst1')) / st1
            delta_st2 = (greeks_up.get('dst2') + greeks_down.get('dst2')) / st2
            gamma_st1 = (greeks_up.get('dst1**2') + greeks_down.get('dst1**2')) / (st1 ** 2)
            gamma_st2 = (greeks_up.get('dst2**2') + greeks_down.get('dst2**2')) / (st2 ** 2)
            x_gamma = (greeks_up.get('dst1*dst2') + greeks_down.get('dst1*dst2')) / (st1 * st2)
            theta = greeks_up.get('dt') + greeks_down.get('dt')

            bt[date] = {
                'business_day_to_expiry': business_day_to_expiry,
                't': t,
                'st1': st1, 'st2': st2,
                'st1_pct': st1_pct, 'st2_pct': st2_pct,
                'pv': pv,
                'delta_st1': delta_st1, 'delta_st2': delta_st2,
                'gamma_st1': gamma_st1, 'gamma_st2': gamma_st2,
                'x_gamma': x_gamma, 'theta': theta
            }

            business_day_to_expiry = business_day_to_expiry - 1
            t = business_day_to_expiry / self.periods_per_year

        # -------
        # COMPUTE REPLICATION STRATEGY AND PNL BREAK-DOWN
        # -------
        bt_df = pd.DataFrame.from_dict(bt, orient='index')

        bt_df.loc[:, 'option_pnl'] = bt_df['pv'].diff()

        bt_df.loc[:, 'delta_st1_pnl'] = bt_df['st1'].diff() * bt_df['delta_st1'].shift()
        bt_df.loc[:, 'delta_st2_pnl'] = bt_df['st2'].diff() * bt_df['delta_st2'].shift()
        bt_df.loc[:, 'delta_pnl'] = bt_df['delta_st1_pnl'] + bt_df['delta_st2_pnl']

        bt_df.loc[:, 'gamma_st1_pnl'] = bt_df['st1'].diff().pow(2) * bt_df['gamma_st1'].shift() * 0.5
        bt_df.loc[:, 'gamma_st2_pnl'] = bt_df['st2'].diff().pow(2) * bt_df['gamma_st2'].shift() * 0.5
        bt_df.loc[:, 'gamma_pnl'] = bt_df['gamma_st1_pnl'] + bt_df['gamma_st2_pnl']

        bt_df.loc[:, 'x_gamma_pnl'] = bt_df['st1'].diff() * bt_df['st2'].diff() * bt_df['x_gamma'].shift()

        bt_df.loc[:, 'explained_pnl'] = bt_df['delta_pnl'] + bt_df['gamma_pnl'] + bt_df['x_gamma_pnl'] + bt_df['theta']

        bt_df.loc[:, 'delta_cumpnl'] = bt_df['delta_pnl'].cumsum()
        bt_df.loc[:, 'option_cumpnl'] = bt_df['option_pnl'].cumsum()

        return bt_df

In [6]:
# -------
# IMPORT DATA
# -------
df1 = pd.read_csv('data/SPY.csv', index_col=0, parse_dates=True).loc[:, ['Adj Close']]
df2 = pd.read_csv('data/IWM.csv', index_col=0, parse_dates=True).loc[:, ['Adj Close']]
df1.columns = ['st1']
df2.columns = ['st2']
df = pd.concat([df1, df2], axis=1).dropna()

In [7]:
df_snap = df.iloc[-20:]
data = df_snap.to_dict(orient='index')

In [8]:
df_snap.pct_change().std() * np.sqrt(252)

st1    0.117115
st2    0.141322
dtype: float64

In [9]:
df_snap.diff().corr()

,st1,st2
st1,1.000000,0.788747
st2,0.788747,1.000000


In [11]:
bt = Backtest(model='numerical_integration')

In [12]:
results = bt.dual_digital(data=data, k1=1, k2=1, iv1=0.12, iv2=0.14, q1=0, q2=0, b1=0, b2=0, rho=0.8, r=0, unit=100)

  0%|          | 0/20 [00:00<?, ?it/s]

In [26]:
results.head()

,business_day_to_expiry,t,st1,st2,st1_pct,st2_pct,pv,delta_st1,delta_st2,gamma_st1,gamma_st2,x_gamma,theta,option_pnl,delta_st1_pnl,delta_st2_pnl,delta_pnl,gamma_st1_pnl,gamma_st2_pnl,gamma_pnl,x_gamma_pnl,explained_pnl,delta_cumpnl,option_cumpnl
2023-09-06,19,0.08,444.65,185.26,1.00,1.00,79.52,-0.44,-0.76,-0.19,-0.81,0.49,-0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-07,18,0.07,443.29,183.47,1.00,0.99,79.27,-1.10,0.02,-0.20,-0.82,0.51,-0.01,-0.25,0.60,1.36,1.96,-0.18,-1.31,-1.48,1.20,1.66,1.96,-0.25
2023-09-08,17,0.07,443.95,183.06,1.00,0.99,78.15,-1.46,0.77,-0.19,-0.77,0.51,-0.07,-1.12,-0.74,-0.01,-0.74,-0.04,-0.07,-0.11,-0.14,-1.07,1.22,-1.37
2023-09-11,16,0.06,446.87,183.50,1.00,0.99,75.20,-1.73,2.08,-0.16,-0.68,0.46,-0.22,-2.95,-4.28,0.34,-3.94,-0.82,-0.07,-0.89,0.65,-4.41,-2.72,-4.32
2023-09-12,15,0.06,444.42,183.32,1.00,0.99,77.87,-1.59,0.90,-0.21,-0.86,0.56,-0.09,2.67,4.25,-0.37,3.88,-0.47,-0.01,-0.48,0.20,3.50,1.16,-1.65


In [22]:
priceable1 = DualDigital(
    st1=100, k1=100, iv1=0.2, q1=0, b1=0, direction1='up',
    st2=100, k2=100, iv2=0.3, q2=0, b2=0, direction2='up',
    rho=0.8, r=0, t=1, unit=100, model='numerical_integration'
)
priceable1.calculate_present_value()
priceable1.calculate_greeks()
pv = priceable1.get_present_value()
greeks = priceable1.get_greeks()

In [23]:
bump1 = 2
bump2 = 3
priceable1 = DualDigital(
    st1=100+bump1, k1=100, iv1=0.2, q1=0, b1=0, direction1='up',
    st2=100+bump2, k2=100, iv2=0.3, q2=0, b2=0, direction2='up',
    rho=0.8, r=0, t=251/252, unit=100, model='numerical_integration'
)
priceable1.calculate_present_value()
priceable1.calculate_greeks()
priceable1.get_present_value(), priceable1.get_greeks()

(38.703420454762856,
 {'dst1': 0.9137233466539385,
  'dst2': 0.6803422720587093,
  'dst1**2': -0.05994930347696936,
  'dst2**2': -0.02771764464171156,
  'dst1*dst2': 0.04209553019052237,
  'dt': 0.01767659240110716})

In [25]:
delta1 = greeks.get('dst1') * bump1
delta2 = greeks.get('dst2') * bump2
delta = delta1 + delta2
gamma1 = greeks.get('dst1**2') * 0.5 * (bump1**2)
gamma2 = greeks.get('dst2**2') * 0.5 * (bump2**2)
gamma = gamma1 + gamma2
x_gamma = greeks.get('dst1*dst2') * (bump1*bump2)
theta = greeks.get('dt')
pnl = delta + gamma + x_gamma + theta
pv_new = pv + pnl
pv, pv_new

(34.839025379718095, 38.70111913627463)